In [1]:
import time
start = time.time()
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import openpyxl

#명시적대기 headless 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.80 Safari/537.36')

#options.add_argument('window_size=1920x1080')
#options.add_argument('--disable-gpu')

LIMIT = 1

In [2]:
def get_stack(info):
    
    d_browser = webdriver.Chrome("/usr/local/bin/chromedriver",chrome_options=options);
    d_browser.implicitly_wait(2)
    
    for e_info in info:
        d_url = 'https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx='+e_info['idx']
        #페이지 접속
        d_browser.get(d_url)
        #오류제거
        wait = WebDriverWait(d_browser,20)
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'.tags')))
        #tag접근
        tag_sec = d_browser.find_element(By.CSS_SELECTOR,'.tags')
        stacks = tag_sec.find_element(By.CSS_SELECTOR,'.scroll')
        tmps = stacks.find_elements(By.TAG_NAME, 'a')
        #모든 태크 모으기
        stack = []
        for tmp in tmps:
            url = tmp.get_attribute('href')
            index = url.find('kewd')
            if(index != -1):
                stack.append(url[index+5:])
        e_info["stack"] = stack

    d_browser.quit()
    return info

In [3]:

def get_info(pages):
    
    global LIMIT
    
    URL = "https://www.saramin.co.kr/zf_user/jobs/list/job-category?cat_mcls=2&sort=RD&page="+str(pages)
    
    browser = webdriver.Chrome("/usr/local/bin/chromedriver",chrome_options = options);	#chromedriver 위치
    browser.get(URL);	# 페이지 열기
    browser.implicitly_wait(2)
    #browser.execute_script("window.scrollTo(0,30)")
    
    wait = WebDriverWait(browser,20)
    element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'.common_recruilt_list')))
    
    if (pages == 1):
        list_ = browser.find_element(By.CSS_SELECTOR,'.common_recruilt_list')
        items = list_.find_elements(By.CSS_SELECTOR,'.list_item')
    else:
        items = browser.find_elements(By.CSS_SELECTOR,'.list_item')


    ann_list = []
    

    for item in items:
            
        #print(item.text)
        #id
        idx = str(item.get_attribute('id')[4:])
        #D_URL = 'https://www.saramin.co.kr/zf_user/jobs/relay/view?rec_idx='+idx
        #stack = []
        #stack = get_stack(D_URL)
        # 회사
        company = item.find_element(By.CSS_SELECTOR,".company_nm").find_element(By.TAG_NAME, 'a').text
    
        # 제목
        title = item.find_element(By.CSS_SELECTOR,".notification_info").find_element(By.TAG_NAME, 'a').text
    
        # 경력
        career = item.find_element(By.CSS_SELECTOR,".career").text
    
        # 학력
        education = item.find_element(By.CSS_SELECTOR,".education").text
    
        # 근무형태
        employment_type = item.find_element(By.CSS_SELECTOR,".employment_type").text
    
        # 근무 위치
        try:
            work_place = item.find_element(By.CSS_SELECTOR,".work_place").text
        except Exception as e:
            place = None
        
        #마감일,등록일
        time = item.find_element(By.CSS_SELECTOR,".deadlines").text.split('\n')
        #마감
        deadline = time[0]
        #등록
        reg_date = time[1]
        #print(reg_date)
        
        #오늘 공고만
        if(reg_date == '(1일 전 등록)'):
            LIMIT = 0
            return ann_list
        info = {'title':title, 'company':company, 'career':career, 'education':education,
                'employment_type' :employment_type,  'workc_place':work_place,
                'idx':idx}#,'deadline': deadline, ,'reg':reg_date}

        ann_list.append(info)
        
    
    
    browser.quit()
    return ann_list


In [4]:
def main():
    global LIMIT
    info = []
    pages = 1   
    while(LIMIT):
        info.extend(get_info(pages))
        pages = pages + 1

    result = get_stack(info)
    return result
    
    

In [5]:
data = main()

/var/folders/8l/z4qr6xns7sb_mbw_25xwx0cm0000gn/T/ipykernel_4548/3603556646.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome("/usr/local/bin/chromedriver",chrome_options = options);	#chromedriver 위치
/var/folders/8l/z4qr6xns7sb_mbw_25xwx0cm0000gn/T/ipykernel_4548/3603556646.py:7: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome("/usr/local/bin/chromedriver",chrome_options = options);	#chromedriver 위치
/var/folders/8l/z4qr6xns7sb_mbw_25xwx0cm0000gn/T/ipykernel_4548/2684362449.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  d_browser = webdriver.Chrome("/usr/local/bin/chromedriver",chrome_options=options);
/var/folders/8l/z4qr6xns7sb_mbw_25xwx0cm0000gn/T/ipykernel_4548/2684362449.py:3: DeprecationWarning: use options instead of chrome_options
  d_browser = webdriver.Chrome("/usr/local/bin/chromedriver",chrome_options=options);


In [6]:
end = time.time()
print(format(end-start))

2234.4762251377106


In [7]:
dataA = pd.DataFrame(data)

In [8]:
dataA.to_excel('220210_selenium.xlsx')